In [1]:
%defaultDatasource jdbc:h2:mem:db

In [2]:
CREATE TABLE Quads (
    ID VARCHAR(10) NOT NULL,
    LAT_MIN DECIMAL(10, 7) NOT NULL,
    LAT_MAX DECIMAL(10, 7) NOT NULL,
    LON_MAX DECIMAL(12, 9),
    LON_MIN DECIMAL(12, 9),
    PRIMARY KEY(ID)
) AS SELECT
    ID,
    LAT_MIN,
    LAT_MAX,
    LON_MAX,
    LON_MIN
FROM CSVREAD('quadsss.csv');

CREATE TABLE Postos 
(
    ID INTEGER,
    CLASSE VARCHAR(60) NOT NULL,
    TIPO VARCHAR(60) NOT NULL,
    LAT DECIMAL(10, 8) NOT NULL,
    LON DECIMAL(10, 8) NOT NULL,
    QUAD VARCHAR(10) NOT NULL,
    PRIMARY KEY(ID),
    FOREIGN KEY(QUAD)
      REFERENCES Quads(Id)
          ON DELETE NO ACTION
          ON UPDATE NO ACTION,
) AS SELECT *
FROM CSVREAD('postos.csv');

CREATE TABLE Postes (
    ID INTEGER,
    LAT DECIMAL(10, 8),
    LON DECIMAL(10, 8),
    QUAD VARCHAR(10),
    PRIMARY KEY(ID)
) AS SELECT *
FROM CSVREAD('postes_ilum.csv');

CREATE TABLE Tipos_Crimes (
    NOME VARCHAR(20),
    PRIMARY KEY(NOME)
) AS SELECT
    NOME
FROM CSVREAD('tipos_crimes.csv');

CREATE TABLE Crimes (
    ID INTEGER,
    ANO_BO DECIMAL(5, 0),
    DATA_OCORRENCIA VARCHAR(21),
    HORA_OCORRENCIA VARCHAR(9) NOT NULL,
    PERIODO VARCHAR(12) NOT NULL,
    LAT DECIMAL(10, 8),
    LON DECIMAL(10, 8),
    TIPO_CRIME VARCHAR(20),
    QUAD VARCHAR(10),
    PRIMARY KEY(ID)
) AS SELECT *
FROM CSVREAD('crimes.csv');

In [3]:
CREATE VIEW Numero_crimes_diarios AS
    SELECT Q.ID, COUNT(*) N_CRIMES_DIARIOS
    FROM Crimes C, Quads Q
    WHERE C.QUAD = Q.ID AND (C.PERIODO = 'PELA MANHÃ' OR C.PERIODO = 'A TARDE')
    GROUP BY Q.ID;

CREATE VIEW Numero_crimes_noturnos AS
    SELECT Q.ID, COUNT(*) N_CRIMES
    FROM Crimes C, Quads Q
    WHERE C.QUAD = Q.ID AND (C.PERIODO = 'A NOITE' OR C.PERIODO = 'DE MADRUGADA')
    GROUP BY Q.ID;

In [ ]:
SELECT NCN.ID, NCN.N_CRIMES
    FROM Numero_crimes_noturnos NCN, Numero_crimes_diarios NCD
    WHERE NCN.ID = NCD.ID
    ORDER BY NCN.ID

In [ ]:
Segunda Query - Efeito dos postes em crimes noturnos

In [ ]:
CREATE VIEW Numero_postes AS
    SELECT Q.ID, COUNT(*) N_POSTES
    FROM Postes P, Quads Q
    WHERE P.QUAD = Q.ID
    GROUP BY Q.ID;

SELECT DISTINCT NCN.ID, NCN.N_CRIMES, NP.N_POSTES
    FROM Numero_crimes_noturnos NCN, Numero_postes NP
    WHERE NCN.ID = NP.ID
    ORDER BY NCN.N_CRIMES

In [5]:
CREATE VIEW Numero_crimes AS
    SELECT Q.ID, COUNT(*) N_CRIMES
    FROM Crimes C, Quads Q
    WHERE C.QUAD = Q.ID
    GROUP BY Q.ID;
    
CREATE VIEW Numero_postos AS
    SELECT Q.ID, COUNT(*) N_POSTOS
    FROM Postos P, Quads Q
    WHERE P.QUAD= Q.ID
    GROUP BY Q.ID;

SELECT NC.ID, NC.N_CRIMES, NP.N_POSTOS
    FROM Numero_crimes NC, Numero_postos NP
    WHERE NC.ID = NP.ID
    ORDER BY NC.N_CRIMES

In [8]:
CREATE VIEW Crimes_noite AS
SELECT C.TIPO_CRIME, COUNT(*) Contagem
    FROM Crimes C
    WHERE C.PERIODO = 'A NOITE'
    GROUP BY C.TIPO_CRIME
    ORDER BY Contagem DESC;

SELECT TIPO_CRIME
    FROM Crimes_noite
    WHERE Contagem = (SELECT MAX(Contagem) FROM Crimes_noite)

ROUBO

In [6]:
SELECT C.TIPO_CRIME, COUNT(*) Contagem
    FROM Crimes C
    WHERE C.QUAD IN (SELECT NP.ID
            FROM Numero_postos NP
            WHERE N_POSTOS = (SELECT MAX(N_POSTOS) FROM Numero_postos))
    GROUP BY C.TIPO_CRIME
    ORDER BY Contagem DESC